# **Geonames**

## Описание проекта
**Цель:**
- Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром







**Задачи:**


- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan


- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)


- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями


**Задачи опционально:**


- возможность настройки количества выдачи подходящих названий (например в параметрах метода)


- коррекция ошибок и опечаток. Например Моченгорск -> Monchegorsk


- хранение в PostgreSQL данных geonames


- хранение векторизованных промежуточных данных в PostgreSQL


- предусмотреть методы для настройки подключения к БД


- предусмотреть метод для инициализации класса (первичная векторизация geonames)


- предусмотреть методы для добавления векторов новых гео названий


**Результат:**


- тетрадка с решением задачи (описание проекта, исследование, методы решения)
- python-скрипт, содержащий функцию (класс), для интеграции в систему Заказчика


**Описание данных**



Используемые таблицы с [`geonames:`](http://download.geonames.org/export/dump/).Таблицы помещены в PostgresSQL для удобства использования.


[`geoname`](https://download.geonames.org/export/dump/allCountries.zip)

| Поле             | Описание                                              | Тип         |
|------------------|-------------------------------------------------------|-------------|
| **geonameid**    | Целочисленный идентификатор записи в базе данных Geonames | `integer`   |
| **name**         | Название географической точки (в UTF-8)               | `varchar(200)` |
| **asciiname**    | Название географической точки в чистом ASCII           | `varchar(200)` |
| **alternatenames** | Перечисленные через запятую ASCII имена, автоматически транслитерированные (удобный атрибут из таблицы `alternatename`) | `varchar(10000)` |
| **latitude**     | Широта в десятичных градусах (WGS84)                 | `double`    |
| **longitude**    | Долгота в десятичных градусах (WGS84)                | `double`    |
| **feature_class**| См. [http://www.geonames.org/export/codes.html](http://www.geonames.org/export/codes.html) | `char(1)`   |
| **feature_code** | См. [http://www.geonames.org/export/codes.html](http://www.geonames.org/export/codes.html) | `varchar(10)` |
| **country_code** | Код страны ISO-3166 из двух букв                       | `char(2)`   |
| **cc2**          | Альтернативные коды страны, перечисленные через запятую, ISO-3166 из двух букв | `varchar(200)` |
| **admin1_code**  | Код FIPS (может быть изменен на код ISO), см. исключения ниже, см. файл `admin1Codes.txt` для отображения этого кода | `varchar(20)` |
| **admin2_code**  | Код для второго административного деления, уезд в США, см. файл `admin2Codes.txt` | `varchar(80)` |
| **admin3_code**  | Код для административного деления третьего уровня      | `varchar(20)` |
| **admin4_code**  | Код для административного деления четвертого уровня    | `varchar(20)` |
| **population**   | Население                                             | `bigint`    |
| **elevation**    | Высота над уровнем моря в метрах                      | `integer`   |
| **dem**          | Цифровая модель рельефа, SRTM3 или GTOPO30, средняя высота области 3''x3'' (примерно 90м х 90м) или 30''x30'' (примерно 900м х 900м) в метрах, целочисленный тип данных. SRTM обработан CGIAR/CIAT. | `integer`   |
| **timezone**     | Идентификатор часового пояса IANA (см. файл `timeZone.txt`) | `varchar(40)` |
| **modification_date** | Дата последнего изменения в формате YYYY-MM-DD     | `date`      |


Таблица [`admin1CodesASCII`](https://download.geonames.org/export/dump/admin1CodesASCII.txt)



| Поле            | Описание                                           | Тип      |
|-------------------|-------------------------------------------------------|-----------|
| concatenated_codes | Код административного деления.                        | `TEXT`     |
| name              | Название административного деления.                    | `TEXT`     |
| ascii_name        | ASCII-представление названия административного деления.| `TEXT`      |
| geonameid         | Целочисленный идентификатор записи в базе данных geonames.| `BIGINT`  |


Таблица [`country_info`](https://download.geonames.org/export/dump/countryInfo.txt)



| Поле               | Описание                                         | Тип                 |
|-----------------------|--------------------------------------------------|---------------------|
| country_code          | Код страны по ISO-3166                            | TEXT                |
| iso3                  | Код страны из трех букв по ISO-3166              | TEXT                |
| iso-numeric           | Числовой код страны по ISO-3166                  | BIGINT              |
| fips                  | Код страны по стандарту FIPS                     | TEXT                |
| country               | Название страны                                  | TEXT                |
| capital               | Столица                                          | TEXT                |
| area(in_sq_km)        | Площадь (в квадратных километрах)                | DOUBLE PRECISION    |
| population            | Население                                        | BIGINT              |
| continent             | Континент                                        | TEXT                |
| tld                   | Домен верхнего уровня страны (TLD)               | TEXT                |
| currency_code         | Код валюты                                       | TEXT                |
| currency_name         | Название валюты                                  | TEXT                |
| phone                 | Телефонный код страны                            | TEXT                |
| postal_code_format    | Формат почтового индекса                          | TEXT                |
| postal_code_regex     | Регулярное выражение для почтового индекса       | TEXT                |
| languages             | Используемые языки                                | TEXT                |
| geonameid             | Целочисленный идентификатор записи в базе данных Geonames | BIGINT       |
| neighbours            | Соседи                                           | TEXT                |
| equivalent_fips_code  | Эквивалентный код FIPS                           | DOUBLE PRECISION    |


## Вступление: Аккорды Магии - Верховное Созвучие Волшебства

Прежде чем взмахнуть волшебной палочкой кода, мы начинаем наше великое заклинание с аккордов магии – импорта библиотек. Это не просто строки кода; это верховное созвучие волшебства, где каждая библиотека - как уникальный инструмент в оркестре магии. С этими звуками мы создадим симфонию проекта "Гео-Магии", перенося нас в мир волшебства и данных.

In [ ]:

import pandas as pd
import numpy as np
import pandas as pd
import re


from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sqlalchemy import create_engine,MetaData
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine, inspect
from sqlalchemy import text
import plotly.express as px
from torch.utils.data import DataLoader
import IPython
from sentence_transformers import SentenceTransformer, util
import torch


from translate import Translator
from langdetect import detect
import transliterate
import numpy as np
import psycopg2 as psy
import pickle



Звёзды и Кристаллы - Волшебные Переменные

Прежде чем начать наше волшебное восхождение в мире данных, мы создаем звёзды и кристаллы - наши неотъемлемые переменные. Каждая переменная - это как звезда на небесах, призывающая свет мудрости, и как кристалл, содержащий в себе волшебную силу данных. С их помощью мы будем лепить наше предсказательное заклинание в проекте "Гео-Магии".


In [2]:
COUNTRY_CODES = ['RU', 'BY', 'KG', 'KZ', 'AM', 'TR', 'RS']
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': 5433,
    'database': 'GEO data',
    'query': {}
}

# Глава 1: Ключи Волшебства - Подход к Решению

Цель проекта была как волшебная карта, позволяющая сопоставить произвольные географические названия с унифицированными именами из geonames. Это было важное задание, выполняемое Карьерным центром, чтобы создать решение, способное провести нас сквозь лабиринт разнообразных географических обозначений.

Наш герой, Эдвард-Кодер, вступил в волшебную схватку с данными geonames, чтобы создать заклинание, способное подбирать наиболее подходящие названия. Все, начиная от Еревана и заканчивая Беларусью, Арменией, Казахстаном и другими странами, стали частью этого великого виртуального квеста.

#### Танец с Буквами - Первый Ключ Волшебства

В нашем волшебном квесте выбраны два ключа для решения загадки географических названий. Первый ключ - использование расстояния Левенштейна. Это основной путь, где мы транслитерируем, ищем совпадения в ASCII и возвращаем нужный формат. Дополнительно, мы можем воспользоваться онлайн переводчиком для коррекции ошибок. Этот путь - наш базовый вариант, точка отсчета, от которой мы будем отталкиваться.

#### Волшебство Векторов - SentenceTransformer

Наш второй ключ - использование SentenceTransformer, векторных представлений текста. Этот метод, с поддержкой предобученной модели LaBSE, дает нам возможность эффективно обрабатывать текстовую информацию о географических объектах. Для улучшения эффективности, модель дообучается на парах основных и альтернативных названий городов для всех доступных языков в geonames.

#### Структура Заклинания - От БД к БД

Вся структура нашего проекта следует логике от БД к БД. Данные подтягиваются из PostgresSQL, проходят через волшебные ключи и сохраняются в БД. Наш волшебный код - класс GeoSearch - подключается к необходимой БД, с возможностью выбора использования Transformer для семантического поиска. Мы можем инициализировать векторизацию и сохранить ее в БД. Класс также предусматривает расширение для населенных пунктов из других стран, добавление их в итоговый результат и управление количеством населения из таблицы geoname.


# Глава 2: Открывая Портал Космической Поисковой Релокации(EDA)

В поиске городов с населением от 15000 человек, наш герой исследовал магию релокации в странах, таких как Турция, Сербия и другие. Косинусная схожесть стала светящимся ключом к выбору наилучших географических названий, как если бы он открывал портал в новый мир.

### SQL запрос 

Окунемся в мир запросов PostgreSQL, там, где слова преобразуются в данные. Наш запрос - это путешествие сквозь страны, волшебно соединяющиеся в мире информации.

В этом захватывающем путешествии, в основную таблицу 'geoname', как в центр вселенной, отправим только те страны, что зовут себя Беларусью, Арменией, Казахстаном, Кыргызстаном, Турцией и Сербией. Здесь важно, чтобы население каждого населенного пункта превышало 15 тысяч жителей - ибо эти места населены жизнью и движением.

Пусть наши таблицы 'country_info', 'alter_name' и 'admin_codes' сливаются вместе, чтобы предоставить нам мудрость названий стран и регионов. Здесь, в объединении таблиц, мы обретаем не просто данные, а истинное слияние информации, что позволит нам увидеть мир шире, чем когда-либо прежде.

In [7]:
# проведем ритуал для отрытия портала в БД
engine = create_engine(URL(**DATABASE))



# В полях таблиц мир космических строк,
# Где данные встречаются в образе манер,
# Слились вдвое, душами светят
# Столбцы geoname с admin_codes целый мир объявляют.

# LEFT JOIN вдохнул жизнь, соединил в сны,
# geoname и admin_codes, два мира, две стороны.
# И страны country_info в мир данных влились,
# Чтобы таблицы целым миром стали для нас здесь.

# gc.population > 15000 и страны из списка,
# Призывают лишь те места, что населены живыми.
# И лишь те места, где буквы в образах сияют,
# В запросе этом мир свой истинный обретают.


# читаем заклинивание на открытие портала в нашу финальную таблицу

query = '''
SELECT 
    gc.geonameid,
    gc.name,
    gc.asciiname,
    gc.alternatenames,
    gc.country AS country_code,
    gc.admin1,
    gc.population,
    ci.country,
    ac.ascii_name AS region
FROM 
    geoname gc
    
LEFT JOIN admin_codes ac ON gc.admin1::TEXT = split_part(ac.concatenated_codes, '.', 2)::TEXT
                         AND gc.country = split_part(ac.concatenated_codes, '.', 1)::TEXT
LEFT JOIN 
    country_info ci ON gc.country = ci.country_code
WHERE 
    gc.population > 15000
    AND gc.country IN ('RU', 'BY', 'KG', 'KZ', 'AM', 'TR', 'RS');
'''

# забираем частичку в свой мир
df = pd.read_sql_query(query, con=engine)
df.sample(10)

,geonameid,name,asciiname,alternatenames,country_code,admin1,population,country,region
1959,9883039,Khoroshevo-Mnevniki District,Khoroshevo-Mnevniki District,"Khoroshevo-Mnevniki,Khoroshyovo-Mnevniki,Khoro...",RU,48,172371,Russia,Moscow
2609,443213,Kilis,Kilis,"Eparchia Kilis,Kilis,Kilis Province,Kilis eana...",TR,90,147919,Turkey,Kilis
1520,1520969,Merke,Merke,"Merke,Merki,Мерке,Меркі",KZ,17,15934,Kazakhstan,Zhambyl
1445,1527974,Keminskiy Rayon,Keminskiy Rayon,"Kemin,Keminskij,Keminskij Rajon,Keminskij rajo...",KG,02,48360,Kyrgyzstan,Chuy
2784,551645,Kayakentskiy Rayon,Kayakentskiy Rayon,"Kajakentskij Rajon,Kajakentskij rajon,Kayakent...",RU,17,54832,Russia,Dagestan
53,1519691,Sarqant,Sarqant,"Sarkand,Sarkanskaya,Sarqan,Sarqant,Сарканд,Сар...",KZ,12510144,76919,Kazakhstan,Jetisu Region
828,8632437,İvrindi İlçesi,Ivrindi Ilcesi,"Ivrindi Ilcesi,İvrindi İlçesi",TR,10,35209,Turkey,Balikesir
2837,530848,Maloyaroslavetskiy Rayon,Maloyaroslavetskiy Rayon,"Malojaroslavetskij Rajon,Maloyaroslavetskiy Ra...",RU,25,49479,Russia,Kaluga Oblast
360,578169,Belaya Glina,Belaya Glina,"Belaja Glina,Belaya Glina,Glina,Белая Глина",RU,38,17997,Russia,Krasnodar Krai
787,324190,Alanya,Alanya,"Alaia,Alaiye,Alan'ja,Alana,Alanija,Alanja,Alan...",TR,07,112969,Turkey,Antalya


Добро пожаловать в удивительный мир EDA, где данные оживают, как яркие краски на палитре художника. Здесь каждая цифра, каждая строка — как загадочный образ, скрывающий за собой удивительные истории. Наши первые шаги в этом мире — это увлекательное путешествие по поиску дубликатов, где мы разгадываем тайны повторяющихся данных. И, как искатели сокровищ, мы осторожно исследуем поля, обнаруживая пропуски — те туманные места, где информация скрыта, и которые мы тщательно освещаем светом анализа.

In [61]:
df.isna().sum()

geonameid           0
name                0
asciiname           0
alternatenames    224
country_code        0
admin1              0
population          0
country             0
region             25
dtype: int64

In [63]:
df[df['alternatenames'].isna()].head(5)

,geonameid,name,asciiname,alternatenames,country_code,admin1,population,country,region
99,865080,Opština Beograd-Rakovica,Opstina Beograd-Rakovica,None,RS,SE,108000,Serbia,Central Serbia
113,11961320,European Russia,European Russia,None,RU,00,110000000,Russia,None
637,556384,Industrial’nyy Rayon,Industrial'nyy Rayon,None,RU,80,108173,Russia,Udmurtiya Republic
1071,2131663,Gorod Magadan,Gorod Magadan,None,RU,44,169501,Russia,Magadan Oblast
1153,482217,Tonshayevskiy Rayon,Tonshayevskiy Rayon,None,RU,51,18730,Russia,Nizhny Novgorod Oblast


In [64]:
df[df['region'].isna()].head(5)

,geonameid,name,asciiname,alternatenames,country_code,admin1,population,country,region
0,174982,Republic of Armenia,Republic of Armenia,"'Amenia,'Aminia,Aamenia,Ac-me-ni-a,Ac-me-ni-a ...",AM,00,2951776,Armenia,None
1,1527747,Kyrgyz Republic,Kyrgyz Republic,"Chirgisia,Ciorgastan,Cirgistan,Cu-ro-gu-xtan,C...",KG,00,6315800,Kyrgyzstan,None
110,11961322,Central,Central,Central Federal District,RU,00,38438600,Russia,None
111,11961321,Northwest,Northwest,Northwestern Federal District,RU,00,13583800,Russia,None
112,11961349,Far East,Far East,Far Eastern Federal District,RU,00,6291900,Russia,None


Среди тайн и скрытых образов мы обнаружили пропуски лишь в альтернативных именах и названиях стран. Наши усилия направлены на удаление этих пропусков, чтобы каждая строка, каждая цифра принесла ясность и достоверность в наше путешествие по данным.Для этого мы смело идем вперед, очищая наши данные от пустоты и повторов. Взяв на себя роль уборщиков этого цифрового мира, мы удаляем все строки, где скрыты пустоты, и избавляемся от зеркальных отражений — дубликатов. Таким образом, мы создаем основу для нашего анализа, где каждая строка и каждая цифра приносят ясность и достоверность в наше путешествие по данным.

In [4]:
df = df[~df['region'].isna()]
df = df[~df['alternatenames'].isna()]
df = df.drop_duplicates()

В этих строках кода я вижу танец данных на бумаге моей воображаемой карты. Здесь, с помощью Plotly, мы создаем полотно для наших цифровых картины — гистограмму, которая становится окном в наш мир данных.

Выбирая магию распределения населения по городам в разных странах, мы создаем колоритный пейзаж из столбцов, каждый со своим оттенком, представляющим уникальную страну. Это картина населения, где каждый столбец — это живой рассказ о населенных пунктах, заставляющий наши глаза сиять открытием и пониманием. В этом мире данных, как в волшебном саду, каждый столбец становится частью удивительного исторического пейзажа.


In [74]:

fig = px.histogram(
    df,
    x='population',
    color='country_code',
    
    title='Распределение городов по численности населения',
    labels={'population': 'Численность населения',
            'count': 'Количество городов'}
)

fig.update_layout(
    xaxis_title='Численность населения',
    yaxis_title='Количество городов',
    legend_title='Код страны',
    font=dict(size=12),
      
    showlegend=True,
)

fig.show()

In [77]:
# Создание столбчатой диаграммы
fig = px.bar(
    df,
    x=df['country'].value_counts().index,  # Страны из столбца 'country'
    # Количество записей для каждой страны
    y=df['country'].value_counts().values,
    labels={'x': 'Страна', 'y': 'Количество записей'},
    title='Количество записей по странам',
    color=df['country'].value_counts().index
)

# Настройка внешнего вида диаграммы
fig.update_layout(
    xaxis_title='Страна',
    yaxis_title='Количество записей',
    font=dict(size=12),
    showlegend=False,  # Не показывать легенду
)

fig.show()

In [6]:
# данные для дообучения трансформера 
df.to_csv(r'C:\data_for_transformer.csv',columns=['name','alternatenames'])

# Глава 3: Заклинание Семантического Поиска - Fine-tune a Pretrained Model


В нашем волшебном квесте мы создаем заклинание, чтобы преобразовать обычные географические названия в мудрые и унифицированные формы. Этот подвиг осуществляется великим магом-Кодером на просторах Kaggle, где доступны силы GPU, чтобы открывать новые горизонты понимания и скорости параллельных вычислений 


### Чтение Заклинания - Свиток Призыва Великого Трансформера  


```python
# Подготовка Книги Знаний - Извлечение Названий

# Сначала мы извлекаем из магической книги географических объектов - `geonames` - список названий и их альтернативные формы.
# Это как открывать страницы старинной карты, где каждое название - это ключ к новому приключению.
geonames_names = geonames['name'].tolist()
geonames_alt_names = [alt_name.split(',')
                      for alt_name in geonames['alternatenames']]

# Танец с Возможностями - Создание InputExamples

# Теперь мы танцуем в магическом круге, создавая список входных примеров.
#  Каждая комбинация основного названия и его альтернатив - это магическое заклинание, призывающее силы для понимания семантических связей.
corpus_list = []
for name, alt in zip(geonames_names, geonames_alt_names):
    for alt_name in alt:
        corpus_list.append(InputExample(

            texts=[name.strip(), alt_name.strip()]))

# Заклинание Семантического Мира - Инициализация Модели

# Теперь пришло время вызвать силы семантического мира. 
# Мы создаем волшебный инструмент - SentenceTransformer - и запускаем заклинание с использованием предварительно обученной модели 'sentence-transformers/LaBSE'.
semantic_model = SentenceTransformer('sentence-transformers/LaBSE')

# Танцевальный Ритуал - Подготовка к Обучению

#Теперь мы начинаем великий танец обучения. 
# Создаем особый портал - DataLoader - и подготавливаем свои заклинания для обучения. 
# Это как подготовка к большому танцу, где каждый InputExample - это шаг вперед к мудрости
train_dataloader = DataLoader(corpus_list, shuffle=True, batch_size=16)

# Заклинание Потерь - Создание МегаЗаклинания

# Теперь мы создаем заклинание потерь - MegaBatchMarginLoss. 
# Это особая формула, которая учит модель видеть глубже и понимать семантику магических сочетаний.
train_loss = losses.MegaBatchMarginLoss(semantic_model)

# Раскрепощение Сил - Заморозка и Разморозка

# Здесь мы освобождаем магические силы модели, замораживая и размораживая нужные части. Это как открытие новых заклинаний в книге магии.
auto_model = semantic_model._first_module().auto_model

# Замораживание 
for param in auto_model.parameters():
    param.requires_grad = False
# Разморозка нужные
for param in auto_model.embeddings.parameters():
    param.requires_grad = True
layer_nums = [0, 1, 10, 11]
for layer_no in layer_nums:
    for param in auto_model.encoder.layer[layer_no].parameters():
        param.requires_grad = True
for param in auto_model.pooler.parameters():
    param.requires_grad = True

# Великий Танец Знаний - Обучение Магии

# Теперь начинается великий танец обучения. 
# Мы проводим волшебный ритуал, вливая знания в модель, чтобы она стала мудрее и понимала семантику.
semantic_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100)

#Сохранение Титана
semantic_model.save(r'C:\new')
```



# Глава 4: Оракулы  GeoSearch - Ключ к Бездонным Названиям 
В мистическом лабиринте кода, где строки ткались как заклинания, Эдвард-Кодер создал GeoSearch - класс, открывающий двери в удивительный мир Базы Данных и географических загадок.

В темных коридорах магии, где переплетаются нити виртуальных миров, раскинулось загадочное царство Geosearch. Его основная задача — создать могучий артефакт, способный связать с собой тайные базы данных. Этот артефакт обладает удивительной способностью расширять список мест обитания и выбирать способ своего великого поиска.

Великий маг Эдвард-Кодер  разработал класс Geosearch, который, как живой, может соединяться с нужными базами данных. Этот класс может расширять свой список известных населенных пунктов и выбирать, каким образом осуществлять свой великий поиск. Он может использовать силу трансформера или предпочесть иные магические методы. В зависимости от этого выбора определяется, каким образом будут извлечены результаты: с помощью расстояния Левенштейна или с использованием семантического поиска по векторам, который также известен как Косинусное сходство.

Великое произведение этого класса включает в себя векторные представления и таблицу результатов, которые можно сохранить в базе данных, избегая тем самым траты времени и вычислительных ресурсов. Идея заключается в том, чтобы хранить векторные массивы NumPy в виде магических строк в таблице numpy_arrays. Уникальный идентификатор, известный как uuid, превращается в ключ, а сам массив сохраняется в столбце np_array_bytes в форме магических байтов.

При вызове метода создания объекта класса необходимо указать параметры, по которым класс соединится с тайными базами данных, включая таблицы из geoname. Также следует указать коды стран, минимальный порог населения в итоговой таблице, и название записи, где хранятся или будут храниться великие векторные представления городов.

Для того чтобы волшебный результат сиял перед вами, передайте любой объект, обладающий порядком, в метод get_geoname(). Тогда Geosearch вернет вам названия населенных пунктов с регионом, страной и id для таблицы geoname. Формат вывода может быть в виде датафрейма или списка волшебных словарей, где каждый словарь представляет собой одну строку в великой таблице. Более подробную магию вы найдете в описании методов этого загадочного класса.

In [26]:

class GeoSearch:
    def __init__(self, database=DATABASE, table_name='search_geoname',
                 create_table=False, use_transformer=True, pretrained_model=r'C:\model',
                 countries=COUNTRY_CODES, population_size=15000,  uuid='corpus_embeddings_array'):
        '''
        Инициализация объекта GeoSearch.

        Параметры:
        - database: Параметры подключения к базе данных (по умолчанию переменная DATABASE).
        - table_name: Название таблицы (по умолчанию 'search_geoname').
        - create_table: Флаг для создания таблицы (по умолчанию False).
        - use_transformer: Флаг использования предобученной модели (по умолчанию True).
        - pretrained_model: Ссылка на предобученную модель.
        - countries: код стран для поиска 
        - population_size: минимальный порог населения
        - uiid: название записи с векторным представлением имен населенных пунктов  

        Внутренние переменные:
        - database: Параметры подключения к базе данных.
        - table_name: Название таблицы.
        - df: DataFrame для хранения данных из таблицы.
        - model: Предобученная модель для трансформера.
        - corpus_embeddings: Массив embeddings.
        - use_transformer: Флаг использования трансформера.

        Действия:
        - Установка соединения с базой данных.
        - Создание таблицы, если указан флаг create_table.
        - Чтение данных из таблицы в DataFrame.
        - Проверка наличия необходимых столбцов в таблице.
        - Инициализация предобученной модели и массива embeddings, если указан флаг use_transformer.
        '''
        # Установка соединения с БД
        self.database = database
        self.database_connection()
        # Устанавливаем значение uuid
        self.uuid = uuid

        # Название таблицы
        self.table_name = table_name
        self.countries = countries  
        self.population_size = population_size

        # Если нужно создать таблицу
        if create_table:
            # Вызов метода для создания таблицы
            self.create_table()
        else:
            # SQL-запрос для выборки данных из таблицы
            self.query = f"SELECT * FROM {self.table_name}"
            # Чтение данных из БД в DataFrame
            self.df = pd.read_sql_query(self.query, con=self.engine)

        # Проверка наличия необходимых столбцов в таблице
        self.check_columns()

        # Использовать ли здесь модель
        if use_transformer:
            # Инициализация предобученной модели
            self.model = SentenceTransformer(pretrained_model)

            # Получение массива embeddings
            self.corpus_embeddings = self.get_embeddings_array()

            # Если массив embeddings пуст, сохранение embeddings
            if self.corpus_embeddings is None:
                self.corpus_embeddings = self.create_corpus_embeddings()
                self.save_embeddings(self.corpus_embeddings)

            # Объявление флага
            self.use_transformer = use_transformer
        else:
            self.use_transformer = use_transformer

    def database_connection(self):
        '''Метод для подключения к БД'''


        # Параметры подключения
        self.db_connect_kwargs = {
            'user': self.database.get('username'),
            'password': self.database.get('password'),
            'host': self.database.get('host'),
            'port': self.database.get('port'),
            'dbname': self.database.get('database')
        }

        # Создаем подключение к базе данных
        self.connection = psy.connect(**self.db_connect_kwargs)

        # Устанавливаем автокоммит
        self.connection.set_session(autocommit=True)

        # Инициализируем курсор
        self.cursor = self.connection.cursor()

        # Создаем SQLAlchemy engine для дополнительных возможностей
        self.engine = create_engine(URL(**self.database))

    def execute_sql_query(self):
        '''Метод выполняет SQL-запрос с возможностью указать страны и население'''

        # SQL-запрос с использованием параметров
        self.query = f'''
            SELECT 
                gc.geonameid,
                gc.name,
                gc.asciiname,
                gc.alternatenames,
                gc.country AS country_code,
                gc.admin1,
                gc.population,
                ci.country,
                ac.ascii_name AS region
            FROM 
                geoname gc
            LEFT JOIN admin_codes ac ON gc.admin1::TEXT = split_part(ac.concatenated_codes, '.', 2)::TEXT
                AND gc.country = split_part(ac.concatenated_codes, '.', 1)::TEXT
            LEFT JOIN country_info ci ON gc.country = ci.country_code
            WHERE 
                gc.population > {self.population_size}
                AND gc.country IN {tuple(self.countries)}
        '''

        

        # Выполнение SQL-запроса и возврат результата в виде DataFrame
        return pd.read_sql_query(self.query, con=self.engine)

    def create_table(self):
        '''
        Метод создает таблицу в БД, используя запрос по региону и населению.
        1. Создание DataFrame с использованием SQL-запроса.
        2. Фильтрация строк, где 'name' является NaN.
        3. Фильтрация строк, где 'asciiname' является NaN.
        4. Заполнение пропущенных значений в столбце 'region' значениями из столбца 'name'.
        5. Запись DataFrame в БД с указанием имени таблицы, индекса, и режима замены (replace, если таблица уже существует).
        '''

        # Создание DataFrame с использованием SQL-запроса
        self.df = self.execute_sql_query()

        # Фильтрация строк, где 'name' является NaN
        self.df = self.df[~self.df['name'].isna()]

        # Фильтрация строк, где 'asciiname' является NaN
        self.df = self.df[~self.df['asciiname'].isna()]

        # Заполнение пропущенных значений в столбце 'region' значениями из столбца 'name'
        self.df['region'].fillna(self.df['name'], inplace=True)

        self.df.to_sql(self.table_name,con=self.engine, index=False, if_exists='replace')

    def check_columns(self):
        '''
        Метод проверяет наличие необходимых столбцов в DataFrame.
        1. Задание списка обязательных столбцов.
        2. Проверка отсутствия каждого обязательного столбца в DataFrame.
        3. Если какие-то столбцы отсутствуют, вызывается исключение ValueError с указанием отсутствующих столбцов.
        '''

        # Список обязательных столбцов
        required_columns = ['geonameid', 'name',
                            'region', 'country', 'asciiname']

        # Столбцы, которые отсутствуют в DataFrame
        missing_columns = [
            col for col in required_columns if col not in self.df.columns]

        # Если есть отсутствующие столбцы, вызвать исключение ValueError
        if missing_columns:
            missing_columns_str = ', '.join(missing_columns)
            raise ValueError(f"Отсутствующие столбцы: {missing_columns_str}. "
                             f"Необходимо наличие всех обязательных столбцов в DataFrame.")

    def get_embeddings_array(self):
        '''
        Метод выполняет SQL-запрос к БД для извлечения массива np_array_bytes из таблицы numpy_arrays, используя заданный uuid.

        Параметры:
        - uuid: Уникальный идентификатор массива (по умолчанию 'corpus_embeddings_array').

        Возвращаемое значение:
        - Восстановленный массив, если он найден; в противном случае, возвращается None.
        '''

        # Установка соединения с БД
        self.database_connection()


        try:
            # Выполняем SQL-запрос для выбора массива np_array_bytes по uuid
            self.cursor.execute(
                """
                SELECT np_array_bytes
                FROM corpus_embeddings
                WHERE uuid=%s
                """,
                (self.uuid,)
            )

            # Получаем результат
            self.result = self.cursor.fetchone()

            # Если результат существует
            if self.result:
                # Восстанавливаем массив из байтового представления
                self.retrieved_array = pickle.loads(self.result[0])

                # Закрываем соединение и курсор
                self.close_connection()

                return self.retrieved_array
            else:
                # Выводим сообщение о том, что массив с указанным uuid не найден
                print(f"Массив {self.uuid} с указанным uuid не найден.")
                return None
        except Exception as e:
            # Обработка исключения, например, вывод ошибки
            print(f"Ошибка при выполнении SQL-запроса: {e}")
            return None
        finally:
            # Закрываем соединение и курсор в блоке finally, чтобы гарантировать их закрытие
            self.close_connection()

    def close_connection(self):
        '''Метод закрывает соединение и курсор с БД.'''
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()

    def create_corpus_embeddings(self):
        '''
        Метод создает embeddings для списка имен (corpus) с использованием предобученной модели.

        Возвращаемое значение:
        - массива embeddings.
        '''

        # Получение списка имен из DataFrame
        corpus = self.df['name'].tolist()

        # Использование модели для векторизации списка имен
        embeddings = self.model.encode(
            corpus, convert_to_tensor=False)

        # Возвращение массива embeddings
        return embeddings

    def save_embeddings(self, array=None, uuid=None, drop_table=True):
        '''
        Метод сохраняет массив в базу данных в виде байтового представления.

        Параметры:
        - array: Массив для сохранения (по умолчанию self.corpus_embeddings).
        - uuid: Уникальный идентификатор массива (по умолчанию None).
        - drop_table: Флаг, определяющий, следует ли удалять таблицу перед сохранением новых данных (по умолчанию True).
        '''

        # Создание подключения к базе данных
        self.database_connection()

        # Создание таблицы corpus_embeddings (удаляется, если drop_table установлен в True)
        if drop_table:
            self.cursor.execute(
                """
                DROP TABLE IF EXISTS corpus_embeddings;
                CREATE TABLE corpus_embeddings (
                    uuid VARCHAR PRIMARY KEY,
                    np_array_bytes BYTEA
                )
                """
            )

        # Вставка массива в базу данных
        self.array = array if (array is not None) and (
            len(array) > 0) else self.corpus_embeddings

        # Использование uuid экземпляра класса, если не предоставлен через параметры метода
        self.uuid = uuid if uuid is not None else self.uuid

        self.cursor.execute(
            """
            INSERT INTO corpus_embeddings(uuid, np_array_bytes)
            VALUES (%s, %s)
            """,
            (self.uuid, pickle.dumps(self.array))
        )

        # Закрытие соединения
        self.close_connection()



    def initialize_model(self, pretrained_model, new_corpus=False, uuid=None, drop_table=True):
        '''
        Инициализация модели SentenceTransformer и массива embeddings.

        Параметры:
        - pretrained_model: Путь или название предобученной модели для SentenceTransformer.
        - new_corpus: Флаг, указывающий, следует ли использовать новый корпус (по умолчанию False).
        - uuid: Уникальный идентификатор массива embeddings (по умолчанию 'corpus_embeddings_array').
        - drop_table: Флаг, определяющий, следует ли удалять таблицу перед сохранением новых данных (по умолчанию True).
        '''

        # Инициализация предобученной модели SentenceTransformer
        self.model = SentenceTransformer(pretrained_model)

        # Получение массива embeddings, если не указано использование нового корпуса
        if not new_corpus:
            # Использование uuid экземпляра класса, если не предоставлен через параметры метода
            self.uuid = uuid if uuid is not None else self.uuid
            self.corpus_embeddings = self.get_embeddings_array()
        else:
            self.corpus_embeddings = self.create_corpus_embeddings()
            self.save_embeddings(self.corpus_embeddings, uuid=uuid, drop_table=drop_table)

        # Установка флага для использования трансформера
        self.use_transformer = True


    def get_lev_distance(self, queries_name, translator=False):
        '''
        Метод находит ближайшие города в столбце 'asciiname' с использованием расстояния Левенштейна.

        Параметры:
        - queries_name: Список запросов (названий городов) для поиска ближайших совпадений.
        - translator: Флаг использования переводчика (по умолчанию False).

        Возвращаемое значение:
        - DataFrame с информацией о ближайших совпадениях для каждого запроса.
        '''

        # Список городов из столбца 'asciiname'
        city_names = self.df['asciiname']

        # DataFrame для хранения результатов
        result_queries_list = []

        for query in queries_name:
            # Определение языка введенного запроса
            detect_language = detect(query)

            # Флаг переводчика
            if not translator:
                # Транслитерация с использованием библиотеки transliterate
                latin_text = transliterate.translit(
                    query, detect_language, "en")
                result = process.extractOne(latin_text, city_names)
            else:
                # Использование переводчика
                translator = Translator(
                    provider="mymemory", from_lang=detect_language, to_lang="en")
                translation = translator.translate(query)
                result = process.extractOne(translation, city_names)

            # Добавление индекса
            answer_indx = self.df[self.df['asciiname'] == result[0]].index[0]
        
            result_queries_list.append({
                'query': query,
                'answer_indx': answer_indx,
                'score': result[1]
            })
            
        result_queries = pd.DataFrame(result_queries_list)

        return result_queries

    def find_similar(self, queries_name, top_k=1):
        '''
        Метод находит наиболее похожие города для заданных запросов.

        Параметры:
        - queries_name: Список запросов (названий городов) для поиска похожих.
        - top_k: Количество наиболее похожих результатов для каждого запроса (по умолчанию 1).

        Возвращаемое значение:
        - DataFrame с информацией о наиболее похожих результатах для каждого запроса.
        '''

        # Ограничение top_k, чтобы не превышать размер корпуса
        top_k = min(top_k, len(self.corpus_embeddings))

        # Создание пустого DataFrame для хранения результатов
        result_queries = pd.DataFrame(
            columns=['query', 'answer_indx', 'score'])

        # Итерация по каждому запросу
        for query in queries_name:
            # Получение эмбеддинга для текущего запроса
            query_embedding = self.model.encode(query, convert_to_tensor=True)

            # Использование cosine-similarity и torch.topk для поиска наивысших 5 оценок
            cos_scores = util.cos_sim(
                query_embedding, self.corpus_embeddings)[0]
            top_results = torch.topk(cos_scores, k=top_k)

            # Создание DataFrame с колонкой 'score'
            some_df = pd.DataFrame(columns=['query', 'answer_indx', 'score'])

            # Вставка массива в столбец 'score'
            some_df['answer_indx'] = np.array(top_results[1])
            some_df['score'] = np.array(top_results[0])

            # Заполнение столбца 'query' значением из запроса на всю длину других колонок
            some_df['query'] = [query] * len(top_results[0])

            # Конкатенация DataFrame с результатами
            result_queries = pd.concat([result_queries, some_df])

        return result_queries

    def get_geoname(self, queries_name, to_dict=False, how='tr',
                                top_k=1, translator=False, save_sql=False, sql_name='result_queries', if_exists='append'):
        '''
        Получение информации о наиболее похожих городах для заданных запросов.

        Параметры:
        - queries_name: Список запросов (названий городов).
        - to_dict: Флаг, указывающий, следует ли возвращать результат в виде словаря (по умолчанию False).
        - how: Способ поиска ('tr' для использования трансформера, 'lev' для расстояния Левенштейна).
        - top_k: Количество наиболее похожих результатов для каждого запроса (по умолчанию 1).
        - translator: Флаг использования переводчика (по умолчанию False).
        - save_sql: Флаг для сохранения результатов в базу данных (по умолчанию False).
        - sql_name: Название таблицы для сохранения результатов (по умолчанию 'result_queries').
        - if_exists: Стратегия при существующих записях в базе данных ('fail', 'replace' или 'append') (по умолчанию 'append').

        Возвращаемое значение:
        - DataFrame или словарь с информацией о наиболее похожих результатах для каждого запроса.
        '''

        if how == 'tr':
            # Проверка наличия модели для трансформера
            if not self.use_transformer:
                raise ValueError("Отсутствует модель, инициализируйте модель для использования")
            else:
                # Получение результатов с использованием трансформера
                result_queries = self.find_similar(queries_name, top_k)
                # Выбор соответствующих данных из DataFrame
                geoname = self.df.loc[result_queries['answer_indx'], ['geonameid', 'name', 'region', 'country']].reset_index(drop=True)
                result_queries = result_queries.reset_index(drop=True)
                geoname['score'], geoname['query'] = result_queries['score'], result_queries['query']

                # Сохранение результатов в базу данных, если флаг установлен
                if save_sql:
                    geoname.to_sql(sql_name, con=self.engine, index=False, if_exists=if_exists)

                # Возвращение данных в формате DataFrame или словаря
                if to_dict:
                    return geoname.to_dict(orient='records')
                else:
                    return geoname
        elif how == 'lev' or not self.use_transformer:
            # Получение результатов с использованием расстояния Левенштейна
            result_queries = self.get_lev_distance(queries_name, translator)
            # Выбор соответствующих данных из DataFrame
            geoname = self.df.loc[result_queries['answer_indx'], ['geonameid', 'name', 'region', 'country']].reset_index(drop=True)
            result_queries = result_queries.reset_index(drop=True)
            geoname['score'], geoname['query'] = result_queries['score'], result_queries['query']

            # Сохранение результатов в базу данных, если флаг установлен
            if save_sql:
                geoname.to_sql(sql_name, con=self.engine, index=False, if_exists=if_exists)

            # Возвращение данных в формате DataFrame или словаря
            if to_dict:
                return geoname.to_dict(orient='records')
            else:
                return geoname
        else:
            # Обработка неверного значения параметра 'how'
            raise ValueError("Указанный способ 'how' не в списке ['lev', 'tr']")

        
     

## Танец Силы: Гармония и Демонстрация GeoSearch

In [27]:
# Создаем экземпляр класса с нужными параметрами 
geo = GeoSearch(database=DATABASE, table_name='first_geoname',
                create_table=False, use_transformer=True, pretrained_model=r'C:\model'
                )

In [27]:
# Нужная таблица
geo.df.sample(10)

,geonameid,name,asciiname,alternatenames,country_code,admin1,population,country,region
1435,312114,Hınıs,Hinis,"Hinis,Hınıs",TR,25,35472,Turkey,Erzurum
249,2017487,Raychikhinsk,Raychikhinsk,"Raicihinsk,Raitchikhinsk,Raitschichinsk,Raitxi...",RU,05,23745,Russia,Amur Oblast
815,8505053,Vostochnoe Degunino,Vostochnoe Degunino,"Vostochnoe Degunino,Восточное Дегунино",RU,48,95000,Russia,Moscow
1641,442301,Batikent,Batikent,"Batikent,Batıkent",TR,68,300000,Turkey,Ankara
616,548602,Kingisepp,Kingisepp,"Jaama,Jamburg,Juama,Kinghisepp,Kingisep,Kingis...",RU,42,50566,Russia,Leningradskaya Oblast'
583,466885,Yeysk,Yeysk,"EIK,Eisk,Ejs'k,Ejsk,Ieisk,Ieïsk,Jeisk,Jejsk,Je...",RU,38,87814,Russia,Krasnodar Krai
157,1516601,Zhitikara,Zhitikara,"Dzhetygara,Dzhetygora,Dzhetygorinskiy Sovkhoz,...",KZ,13,43104,Kazakhstan,Qostanay
445,527888,Medvezh’yegorsk,Medvezh'yegorsk,"Karhumaegi,Karhumaeki,Karhumägi,Karhumäki,Kond...",RU,28,16551,Russia,Karelia
1673,306041,Kozluk,Kozluk,Kozluk,TR,76,29502,Turkey,Batman
1566,738349,Ünye,UEnye,"Oenoe,Onieh,UEnye,Un'e,Unia,Unie,Unieh,Ünia,Ün...",TR,52,77585,Turkey,Ordu


In [18]:
# Наличие сохраненных эмбеддингов в бд 
geo.get_embeddings_array()

array([[ 0.03690946, -0.05967313, -0.04160984, ..., -0.03570218,
         0.02268557, -0.0296363 ],
       [-0.02679154,  0.04495007,  0.02939451, ...,  0.00668902,
         0.04198096,  0.01875285],
       [ 0.0405371 , -0.04587199,  0.00842814, ...,  0.02578129,
         0.02500362, -0.04545236],
       ...,
       [-0.04334595,  0.01864237, -0.03222156, ...,  0.03039592,
        -0.04163978, -0.00408723],
       [-0.02191513,  0.01090946,  0.00149709, ..., -0.04565169,
         0.00625616,  0.02827862],
       [-0.00126288, -0.03331743, -0.05996392, ..., -0.0574507 ,
        -0.0544572 , -0.05066687]], dtype=float32)

In [48]:
geo.get_geoname(['Москва', 'Минск'], how='tr')

,geonameid,name,region,country,score,query
0,524901,Moscow,Moscow,Russia,0.960691,Москва
1,625144,Minsk,Minsk City,Belarus,0.918557,Минск


In [47]:
geo.get_geoname(['Москва', 'Минск'], how='lev', to_dict=True)

[{'geonameid': 465057,
  'name': 'Zamoskvorech’ye',
  'region': 'Moscow',
  'country': 'Russia',
  'query': 'Москва',
  'score': 75},
 {'geonameid': 625144,
  'name': 'Minsk',
  'region': 'Minsk City',
  'country': 'Belarus',
  'query': 'Минск',
  'score': 100}]

# Глава 5: Испытание Силы - Путь к Точности GeoSearch

В самых глубинах волшебного леса, где свет проникает лишь сквозь густую листву, развернулась загадочная глава под названием "Испытание Силы: Путь к Точности GeoSearch". На поляне магии, окруженной древними деревьями, началась мистическая церемония проверки могущества класса GeoSearch.

Великий маг, находящийся в центре волшебной арены, провозгласил начало испытания. Силы GeoSearch должны были быть измерены, а точность их великого поиска проверена с использованием загадочной метрики, известной как accuracy. Магия Левенштейна, тонкая настройка трансформера и его ванильная версия — все они были вызваны на свет для демонстрации своих результатов обучения.

Темные заклинания точности пронзали воздух, а маги сравнивали расстояние Левенштейна, витавшее вокруг как тень прошлого, с танцем fine-tune трансформера, ожившим в потоках магии. Ванильная версия трансформера, словно древний заклинатель, представляла свою силу величественным образом.

Для таинственных испытаний использовался древний тестовый датасет, насыщенный запросами, как загадочными посланиями, призывающими ответы из таинственных глубин.

Так развернулась эта загадочная глава, где силы боролись, а точность их проявления становилась ярким светом во мраке.


In [5]:
test_df = pd.read_csv(
    r'C:\geo_test.csv', sep=';')
test_df.head(10)

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan
3,Москва,Moscow,Moscow,Russia
4,Алматы,Almaty,Almaty,Kazakhstan
5,Оренбург,Orenburg,Orenburg Oblast,Russia
6,Новосибирск,Novosibirsk,Novosibirsk Oblast,Russia
7,Кострома,Kostroma,Kostroma Oblast,Russia
8,Ёшкар-Ола,Yoshkar-Ola,Mariy-El Republic,Russia
9,Йошкар-Ола,Yoshkar-Ola,Mariy-El Republic,Russia


In [6]:
# Функция для проверки точности решения 
def accuracy(test, result):
    score = (test['name']==result['name']).mean()
    return score

In [28]:
import time

# Создаем пустой датафрейм
result_df = pd.DataFrame(columns=['Method', 'Accuracy', 'Execution Time'])

# Функция для выполнения get_geoname и измерения времени


def run_and_measure(method, how):
    start_time = time.time()
    result = geo.get_geoname(
        test_df['query'], how=how)
    execution_time = time.time() - start_time
    acc = accuracy(test_df, result)
    result_df.loc[len(result_df)] = [method, acc, execution_time]


# Первый метод: расстояние Левенштейна
run_and_measure('lev', 'lev')

# Второй метод: дообученный трансформер
run_and_measure('tune_tr', 'tr')

# Третий метод: ванильная версия трансформера
geo.initialize_model(pretrained_model='sentence-transformers/LaBSE',
                     new_corpus=False, uuid='vanilla_embeddings', drop_table=False)
run_and_measure('vanilla_tr', 'tr')

result_df

,Method,Accuracy,Execution Time
0,lev,0.857971,119.522237
1,tune_tr,0.924638,39.578913
2,vanilla_tr,0.863768,41.049026


### Триумф Fine-Tune: Вечное Сияние Времени и Точности

В склоненных к таинственным измерениям времени, где свет и тьма танцевали свой вечный танец, раздался мудрый шепот о победе fine-tune трансформера. Звезды, свидетели этой великой битвы точности, сияли сильнее, вручая триумфатору свою благосклонность. Работа, проведенная в иссушающих потоках магии, оправдала себя, и fine-tune трансформер поднялся на вершину.

Ванильная версия трансформера, словно великолепный хранитель старых тайн, предстала в свете своих успехов, чуть превосходя расстояние Левенштейна. Ее светлый свет являлся нежным ответом на вызов темных сил.

Однако мудрый маг напомнил, что время выполнения этого великого танца зависит от могущества машины, на которой разворачиваются магические вычисления. Так, вечная борьба точности и времени оставалась вечным вопросом, определяемым могучими силами машинной магии.

Так закончилась эта загадочная глава, где триумф и тень переплелись в вечном плясе времени и точности.

# Эпилог 

В закатном свете, когда тени заключают в себе тайны и великие магии пробуждаются, начинается последний акт саги Geosearch. Великий маг, стоящий перед вратами базы данных, восклицает:

"Для того чтобы заложить камень основания географического волшебства, была совершена тщательная работа. Построена Башня PostgresSQL, куда вложены не только камни данных, но и первичные взгляды в волшебство гео-семантики."
#### Шаг 1: Сила Базы
Среди заклинаний баз данных было выполнено построение PostgresSQL. Там, где каждый столбец был словно камень мудрости, заложенный для хранения данных, проведен первичный анализ, раскрывающий тайны потока информации.
#### Шаг 2: Магия Fine-Tune
"Великий трансформер, погруженный в волшебные потоки дообучения, взошел в своей силе. Его магия стала ключом к семантическому поиску, открывая двери в неизведанные земли значений."
#### Шаг 3: Класс Geosearch
"Создан магический класс Geosearch, который соединяет миры данных и географии. Семантический поиск и расстояние Левенштейна — два пути, по которым великая магия производит свои результаты, сохраняя их в свитках базы данных."
#### Шаг 4: Испытание Магии
"Эпические испытания Geosearch оставили отпечатки во времени. Тестирование класса стало великим сражением, где точность и великий поиск вступили в магическое противостояние."

#### Рекомендации Магистра
"Советы великого мага становятся ключами к дальнейшему восхождению. Fine-tune модели — волшебная сущность, которую следует питать новыми корпусами текстов. Внедрение коррекции опечаток станет заклинанием точности, а возможность запланированного дообучения — переходом в новые страны и языки."

Так завершается эта загадочная глава, где рекомендации магистра становятся светилами на пути в мире гео-волшебства.